In [ ]:
import json
from pathlib import Path

from Libraries import Common_Utils as UTL
from Libraries import Common_Helpers as helpers
from Libraries import Processor_Datasets as ds_proc
from Libraries import Processor_Models as model_proc
from Libraries import Flow_Reasoning as flow_reason
from Libraries import Flow_Critical as flow_critic
from Libraries.exceptions import PipelineAbortSample

In [ ]:
LANG = "en"
LANG_UPPER = LANG.upper()

HISTORIES = Path("Data/Histories.json")
CONFIG_PATH = Path("Config/config.json")
BASE_MODELS_DIR = Path("Models")
BASE_DATA_DIR = Path("Data")

HISTORIES.parent.mkdir(parents=True, exist_ok=True)
PRE_JSON_FILE = f"Output/output-{LANG_UPPER}-merged.json"
BATCH_HISTORY_FILE = BASE_DATA_DIR / f"Histories-Batch-{LANG_UPPER}-test.json"

REASON_CLIENT = None
CRITIC_CLIENT = None
HF_DATASET = None
NO_REASON = ""
NO_CRITIC = ""
FIRST_REASON_PATH = ""
REFINE_REASON_PATH = ""
FIRST_CRITIC_PATH = ""
REFINE_CRITIC_PATH = ""
FIRST_REASON = ""
REFINE_REASON = ""
FIRST_CRITIC = ""
REFINE_CRITIC = ""
CONFIG = {}
REASON_PARAMS = {}
CRITIC_PARAMS = {}
LLAMA_CPP_PARAMS = {}
FLOW_PARAMS = {}

CONFIG = UTL.read_json(CONFIG_PATH)

if CONFIG:
    NO_REASON_PATH = Path(CONFIG["paths"][f"prompt_reason_no-{LANG}"])
    # NO_CRITIC_PATH = Path(CONFIG["paths"][f"prompt_critic_no-{LANG}"])
    FIRST_REASON_PATH = Path(CONFIG["paths"][f"prompt_reason_first-{LANG}"])
    REFINE_REASON_PATH = Path(CONFIG["paths"][f"prompt_reason_refine-{LANG}"])
    FIRST_CRITIC_PATH = Path(CONFIG["paths"][f"prompt_critic_first-{LANG}"])
    REFINE_CRITIC_PATH = Path(CONFIG["paths"][f"prompt_critic_refine-{LANG}"])

    NO_REASON = UTL.read_text(NO_REASON_PATH)
    # NO_CRITIC = UTL.read_text(NO_CRITIC_PATH)
    FIRST_REASON = UTL.read_text(FIRST_REASON_PATH)
    REFINE_REASON = UTL.read_text(REFINE_REASON_PATH)
    FIRST_CRITIC = UTL.read_text(FIRST_CRITIC_PATH)
    REFINE_CRITIC = UTL.read_text(REFINE_CRITIC_PATH)

    REASON_PARAMS = CONFIG.get("reason_params", {})
    CRITIC_PARAMS = CONFIG.get("critic_params", {})
    LLAMA_CPP_PARAMS = CONFIG.get("llama_cpp_params", {})
    FLOW_PARAMS = CONFIG.get("flow_params", {})

    print("✅ Tất cả file cấu hình và prompt đã tải thành công.")

In [ ]:
if CONFIG:
    dataset_config = CONFIG[f"dataset-{LANG}"]
    parts = dataset_config['name'].split('/')
    publisher = parts[0] if len(parts) > 1 else 'default'
    dataset_name = parts[-1]
    local_path = BASE_DATA_DIR / publisher / dataset_name
    
    if local_path.exists():
        print(f"✅ Đang tải dataset cục bộ từ: {local_path.resolve()}")
        HF_DATASET = ds_proc.load_from_disk_internal(local_path)
    else:
        print(f"⚠️ Không tìm thấy dataset cục bộ. Đang tải về từ '{dataset_config['name']}'...")
        HF_DATASET = ds_proc.download_and_save_internal(dataset_config, local_path)
        
    if not HF_DATASET:
            print("❌ Tải dataset thất bại.")
else:
    print("⛔ Config không được tải. Bỏ qua Giai đoạn 2.")

In [ ]:
if CONFIG: 
    REASON_CLIENT, CRITIC_CLIENT = model_proc.llm_initialize(
        config=CONFIG,
        llama_cpp_params=LLAMA_CPP_PARAMS,
        base_models_dir=BASE_MODELS_DIR
    )

In [ ]:
if HF_DATASET:
    print("\n--- 📊 Phân tích Dataset ---")
    analysis = ds_proc.analyze_dataset_internal(HF_DATASET)
    if "error" in analysis:
        print(f"Lỗi phân tích: {analysis['error']}")
    else:
        print(f"Số lượng mẫu tổng: {analysis['count']}")
        print(f"Cấu trúc (Features): {analysis['features']}")
        print(f"Tìm thấy 'article': {analysis['has_article']}")
        print(f"Tìm thấy 'summary': {analysis['has_summary']}")
    print("."*50)
    
    INDEX_START = 0
    INDEX_END = analysis['count'] - 1

    if INDEX_END >= analysis['count']:
        print(f"⚠️ Cảnh báo: INDEX_END ({INDEX_END}) vượt quá số lượng mẫu ({analysis['count']}).")
        print(f"Sẽ tự động điều chỉnh INDEX_END về {analysis['count'] - 1}.")
        INDEX_END = analysis['count'] - 1
    
    print(f"\n✅ Sẵn sàng chạy quy trình cho các mẫu từ {INDEX_START} đến {INDEX_END}.")

else:
    print("⛔ Dataset chưa được tải, không thể tiếp tục.")

In [ ]:
def mainFlow(source_text: str, max_iters: int, min_improve: float) -> dict:
    
    best_reasoning_json = None
    best_score = 0.0
    history_log = {
        "source_text": source_text,
        "iterations": []
    }
    current_feedback = None
    last_reasoning_json = ""
    critical_output = {}

    for step in range(0, max_iters + 1):
        
        if step == 0:
            first_reason = NO_REASON
            first_critic = NO_CRITIC
        else:
            first_reason = FIRST_REASON
            first_critic = FIRST_CRITIC
            if step == 1:
                current_feedback = None
                critical_output = {}
            
        print(f"\n🔄 Vòng {step} ...")

        reasoning_json = flow_reason.run(
            client=REASON_CLIENT,
            reason_prompt=first_reason,
            refine_prompt=REFINE_REASON,
            generation_params=REASON_PARAMS,
            source_text=source_text,
            current_reasoning=last_reasoning_json,
            feedback=current_feedback,
        )
        
        if not reasoning_json or not str(reasoning_json).strip():
            print("⛔ Lỗi: Reasoning trả về rỗng, dừng vòng lặp.")
            break
            
        print(f"\n🔄 Reaoning Result:\n{reasoning_json}")

        last_reasoning_json = reasoning_json

        critical_output = flow_critic.run(
            client=CRITIC_CLIENT,
            critic_prompt=first_critic,
            refine_prompt=REFINE_CRITIC,
            generation_params=CRITIC_PARAMS,
            source_text=source_text,
            reasoning_output=reasoning_json,
            prev_result=critical_output,
        )
        
        if "error" in critical_output:
            print(f"⛔ Lỗi từ Critical: {critical_output['error']}")
            if "raw_response" in critical_output:
                print("--- RAW OUTPUT ---")
                print(critical_output["raw_response"])
                print("------------------")
            history_log["iterations"].append({"round": step, "error": critical_output})
            break
        
        average_score = helpers.average_score(critical_output)
        current_feedback = critical_output.get("feedback_text", "")

        print(f"📊 Điểm TBC: {average_score:.2f}")
        print(f"📝 Nhận xét (Toàn bộ JSON): {json.dumps(critical_output, ensure_ascii=False, indent=2)}\n")

        history_log["iterations"].append({
            "round": step, 
            "article:": source_text,
            "reasoning": reasoning_json,
            "evaluation": critical_output.get("scoring", {}),
            "average_score": average_score,
            "feedback": current_feedback
        })

        # early exit rules
        if average_score > 4.8:
            best_reasoning_json, best_score = reasoning_json, average_score
            print("✅ Kết quả tốt, dừng sớm")
            break


        if best_score == 0:  # first run
            best_reasoning_json, best_score = reasoning_json, average_score
            print(f"📈 Lần đầu, điểm: {best_score:.2f}")
        elif (average_score - best_score) > min_improve:
            best_reasoning_json, best_score = reasoning_json, average_score
            print(f"📈 Cải thiện tốt, điểm mới: {best_score:.2f}")
        elif (average_score - best_score) >= 0:
            best_reasoning_json, best_score = reasoning_json, average_score
            print(f"⛔ Không cải thiện đáng kể, điểm mới: {best_score:.2f}")
        else:
            best_reasoning_json, best_score = reasoning_json, average_score
            print(f"⛔ Kết quả giảm sút, điểm mới: {best_score:.2f}")

    return {
        "best_reasoning": best_reasoning_json,
        "best_score": best_score,
        "history": history_log
    }

In [ ]:
successful_runs = 0

INDEX_START = 0

if LANG == "vi":
    INDEX_END = 390
else:
    INDEX_END = 495
    
print(f"🚀 BẮT ĐẦU CHẠY HÀNG LOẠT CHO {INDEX_END - INDEX_START + 1} MẪU... ({INDEX_START} → {INDEX_END})")
print(f"Kết quả → {BATCH_HISTORY_FILE.resolve()}")
print("="*70)

sumaryData = UTL.read_json(PRE_JSON_FILE)
print(PRE_JSON_FILE)

for i in range(INDEX_START, INDEX_END + 1):
    print(f"\n\n--- 🔄 Mẫu #{i} ---")
    # raw_text = ds_proc.get_content_by_index_internal(HF_DATASET, i)

    raw_text = sumaryData[f"index_{i}"]["rounds"][0]["article:"]
   
    if not raw_text:
        print(f"⛔ Không lấy được article cho index {i}")
        continue

    # --- TIỀN XỬ LÝ CHUẨN ---
    text = raw_text.replace("\n", " ")        # bỏ xuống dòng
    text = " ".join(text.split())             # normalize space

    input_article = text

    try:
        print(f"Bắt đầu mainFlow → {FLOW_PARAMS}")
        result = mainFlow(
            input_article,
            max_iters=0, #FLOW_PARAMS.get("max_iters", 3),
            min_improve=FLOW_PARAMS.get("min_improve", 0.05)
        )

        history_key = f"index_{i}"
        history_data = result["history"].get("iterations", [])

        UTL.update_json_dict(history_key, history_data, BATCH_HISTORY_FILE, indent=2)
        print(f"✅ Lưu xong {history_key}")
        successful_runs += 1

        print("\n" + "."*50)
        print(f"🎯 Điểm: {result['best_score']}")
        print(f"🧠 Tóm tắt: {result['best_reasoning']}")
        print("."*50)

    except PipelineAbortSample as e:
        print(f"⚠️ Skip mẫu {i}: {e}")
        UTL.update_json_dict(f"index_{i}", {"status":"skipped","error":str(e)}, BATCH_HISTORY_FILE, indent=2)
        continue

    except Exception as e:
        print(f"❌ Fatal lỗi tại index {i}: {e}")
        UTL.update_json_dict(f"index_{i}", {"status":"fatal","error":str(e)}, BATCH_HISTORY_FILE, indent=2)
        raise

print("\n" + "="*70)
print("✅ HOÀN TẤT")
print(f"🎉 Thành công: {successful_runs} mẫu")
print(f"📦 Log: {BATCH_HISTORY_FILE.resolve()}")
